<a href="https://colab.research.google.com/github/21-LAKSHMI/AI-AND-ML/blob/main/CV_Decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [8]:
# importing decision tree classifier
from sklearn.tree import DecisionTreeClassifier

In [9]:
from sklearn.model_selection import GridSearchCV, cross_val_predict

# Define parameter grid for DecisionTreeClassifier
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],  # splitting criterion
    'splitter': ['best', 'random'],               # strategy to choose split
    'max_depth': [None, 5, 10, 20],               # depth of tree
    'min_samples_split': [2, 5, 10],              # minimum samples to split
    'min_samples_leaf': [1, 2, 5],                # minimum samples per leaf
    'max_features': [None, 'sqrt', 'log2']        # features considered for best split
}

# Initialize GridSearchCV with DecisionTreeClassifier
grid = GridSearchCV(
    DecisionTreeClassifier(),
    param_grid,
    refit=True,
    verbose=3,
    n_jobs=-1,
    scoring='f1_weighted'
)

# Fit the model
grid.fit(X, y)


Fitting 5 folds for each of 648 candidates, totalling 3240 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 5, 10, 20],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [10]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [11]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [12]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'criterion': 'log_loss', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 5, 'splitter': 'random'}
The f1_macro: 0.863705198299054
The confusion Matrix:
 [[223  34]
 [ 21 122]]
The report:
               precision    recall  f1-score   support

           0       0.91      0.87      0.89       257
           1       0.78      0.85      0.82       143

    accuracy                           0.86       400
   macro avg       0.85      0.86      0.85       400
weighted avg       0.87      0.86      0.86       400



In [13]:
# Save Best Model
import pickle
filename = "SVC_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [14]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002979,0.000601,0.004721,0.000570,gini,None,None,1,2,best,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.716667,0.887013,0.829164,0.766960,0.779316,0.795824,0.057963,548
1,0.002192,0.000172,0.003623,0.000098,gini,None,None,1,2,random,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.798551,0.885893,0.859227,0.777666,0.779316,0.820131,0.044245,387
2,0.002349,0.000115,0.003880,0.000124,gini,None,None,1,5,best,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.716667,0.872507,0.857479,0.803611,0.803125,0.810678,0.054696,458
3,0.002101,0.000078,0.004633,0.000672,gini,None,None,1,5,random,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.813654,0.924319,0.847275,0.828159,0.773146,0.837310,0.049860,234
4,0.002501,0.000432,0.003932,0.000060,gini,None,None,1,10,best,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.783072,0.922545,0.855447,0.803611,0.681234,0.809182,0.080094,469


In [16]:

# Get user input for Age, EstimatedSalary, and Gender
age = int(input("Enter Age: "))
estimated_salary = int(input("Enter Estimated Salary: "))
gender = input("Enter Gender (Male/Female): ")

# Convert gender to numerical representation (1 for Male, 0 for Female)
gender_male = 1 if gender.lower() == 'male' else 0

# Create a numpy array from user input
user_input = np.array([[age, estimated_salary, gender_male]])

# Scale the user input using the same StandardScaler fitted on the training data
scaled_input = sc.transform(user_input)

# Make prediction using the best model
prediction = grid.best_estimator_.predict(scaled_input)

# Display the prediction.
if prediction[0] == 1:
    print("Prediction: Purchased")
else:
    print("Prediction: Not Purchased")

Enter Age: 46
Enter Estimated Salary: 41000
Enter Gender (Male/Female): Female
Prediction: Purchased


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
